# Ronen Segev

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

## נבדקים

In [ ]:
nivdakim_df = pd.read_csv("./שאלון לנבדק _מעבדה בפיזיולוגיה של החי_.csv").rename(columns={
    "מספר נבדק": "participant",
    "מין": "sex",
    "מעשן/נת או לא מעשן/נת?": "is_smoker",
    "מתאמן/נת או לא מתאמן/נת? ": "is_excercising",
}).drop(columns=["חותמת זמן", "גיל"])

nivdakim_df["is_smoker"] = nivdakim_df["is_smoker"].apply(lambda x: x == "כן")
nivdakim_df["is_excercising"] = nivdakim_df["is_excercising"].apply(lambda x: x == "כן")
nivdakim_df["is_male"] = nivdakim_df["sex"].apply(lambda x: x == "זכר")
nivdakim_df = nivdakim_df.drop(columns=["sex"])

smokers = nivdakim_df[nivdakim_df["is_smoker"]]["participant"].tolist()
excercisers = nivdakim_df[nivdakim_df["is_excercising"]]["participant"].tolist()
males = nivdakim_df[nivdakim_df["is_male"]]["participant"].tolist()

## Parse data

In [ ]:
# Define the folder containing the files
folder_path = "./results/"

# Initialize an empty DataFrame
unified_data = pd.DataFrame()

# Process each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        # Extract participant and file type from file names
        participant, file_type = file_name.split(".")[0], file_name.split(".")[1]

        # Read the file into a DataFrame
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, comment="#", header=None, names=["Marker ID", "bpm (s)"])

        # Add metadata columns
        df["Participant"] = participant
        df["File Type"] = file_type

        # Append to the main DataFrame
        unified_data = pd.concat([unified_data, df], ignore_index=True)

# Save the unified DataFrame to a CSV or display it
unified_data.to_csv("unified_results.csv", index=False)
print("Unified data has been saved to 'unified_results.csv'.")

Unified data has been saved to 'unified_results.csv'.


## Filter unnecessary rows and cols

In [ ]:
df = pd.read_csv("./unified_results.csv")

df = df.rename(columns={
    "Participant": "participant",
    "Marker ID": "marker_id",
    "bpm (s)": "bpm",
    "File Type": "file_type"
})

df["participant"] = df["participant"].astype(int)

df = df[df["marker_id"] == "_ch0_neuron0"]

In [ ]:
bs_df = df[df["file_type"] == "bs-events"]
exp_df = df[df["file_type"] == "exp-events"]

bs_df = bs_df.drop(columns=["marker_id", "file_type"], errors="ignore")
exp_df = exp_df.drop(columns=["marker_id", "file_type"], errors="ignore")

assert len(bs_df) + len(exp_df) == len(df)

In [ ]:
contr_before_run = bs_df[(bs_df["bpm"] < 15) & (bs_df["participant"] % 2 == 0)].groupby("participant").count() * 4
contr_recovery_0 = exp_df[(exp_df["bpm"] < 15) & (exp_df["participant"] % 2 == 0)].groupby("participant").count() * 4
contr_recovery_1 = exp_df[(exp_df["bpm"] >= 15) & (exp_df["bpm"] < 30) & (exp_df["participant"] % 2 == 0)].groupby("participant").count() * 4
contr_recovery_2 = exp_df[(exp_df["bpm"] >= 30) & (exp_df["bpm"] < 45) & (exp_df["participant"] % 2 == 0)].groupby("participant").count() * 4
contr_recovery_3 = exp_df[(exp_df["bpm"] >= 45) & (exp_df["bpm"] < 60) & (exp_df["participant"] % 2 == 0)].groupby("participant").count() * 4

treat_before_run = bs_df[(bs_df["bpm"] < 15) & (bs_df["participant"] % 2 == 1)].groupby("participant").count() * 4
treat_recovery_0 = exp_df[(exp_df["bpm"] < 15) & (exp_df["participant"] % 2 == 1)].groupby("participant").count() * 4
treat_recovery_1 = exp_df[(exp_df["bpm"] >= 15) & (exp_df["bpm"] < 30) & (exp_df["participant"] % 2 == 1)].groupby("participant").count() * 4
treat_recovery_2 = exp_df[(exp_df["bpm"] >= 30) & (exp_df["bpm"] < 45) & (exp_df["participant"] % 2 == 1)].groupby("participant").count() * 4
treat_recovery_3 = exp_df[(exp_df["bpm"] >= 45) & (exp_df["bpm"] < 60) & (exp_df["participant"] % 2 == 1)].groupby("participant").count() * 4

In [ ]:
contr_before_run = contr_before_run.reset_index()
contr_recovery_0 = contr_recovery_0.reset_index()
contr_recovery_1 = contr_recovery_1.reset_index()
contr_recovery_2 = contr_recovery_2.reset_index()
contr_recovery_3 = contr_recovery_3.reset_index()
treat_before_run = treat_before_run.reset_index()
treat_recovery_0 = treat_recovery_0.reset_index()
treat_recovery_1 = treat_recovery_1.reset_index()
treat_recovery_2 = treat_recovery_2.reset_index()
treat_recovery_3 = treat_recovery_3.reset_index()

smk_contr_before_run = contr_before_run[contr_before_run["participant"].isin(smokers)] 
smk_contr_recovery_0 = contr_recovery_0[contr_recovery_0["participant"].isin(smokers)] 
smk_contr_recovery_1 = contr_recovery_1[contr_recovery_1["participant"].isin(smokers)] 
smk_contr_recovery_2 = contr_recovery_2[contr_recovery_2["participant"].isin(smokers)] 
smk_contr_recovery_3 = contr_recovery_3[contr_recovery_3["participant"].isin(smokers)] 
no_smk_contr_before_run = contr_before_run[~contr_before_run["participant"].isin(smokers)]
no_smk_contr_recovery_0 = contr_recovery_0[~contr_recovery_0["participant"].isin(smokers)]
no_smk_contr_recovery_1 = contr_recovery_1[~contr_recovery_1["participant"].isin(smokers)]
no_smk_contr_recovery_2 = contr_recovery_2[~contr_recovery_2["participant"].isin(smokers)]
no_smk_contr_recovery_3 = contr_recovery_3[~contr_recovery_3["participant"].isin(smokers)]
smk_treat_before_run = treat_before_run[treat_before_run["participant"].isin(smokers)]
smk_treat_recovery_0 = treat_recovery_0[treat_recovery_0["participant"].isin(smokers)]
smk_treat_recovery_1 = treat_recovery_1[treat_recovery_1["participant"].isin(smokers)]
smk_treat_recovery_2 = treat_recovery_2[treat_recovery_2["participant"].isin(smokers)]
smk_treat_recovery_3 = treat_recovery_3[treat_recovery_3["participant"].isin(smokers)]
no_smk_treat_before_run = treat_before_run[~treat_before_run["participant"].isin(smokers)]
no_smk_treat_recovery_0 = treat_recovery_0[~treat_recovery_0["participant"].isin(smokers)]
no_smk_treat_recovery_1 = treat_recovery_1[~treat_recovery_1["participant"].isin(smokers)]
no_smk_treat_recovery_2 = treat_recovery_2[~treat_recovery_2["participant"].isin(smokers)]
no_smk_treat_recovery_3 = treat_recovery_3[~treat_recovery_3["participant"].isin(smokers)]

excercise_contr_before_run = contr_before_run[contr_before_run["participant"].isin(excercisers)] 
excercise_contr_recovery_0 = contr_recovery_0[contr_recovery_0["participant"].isin(excercisers)] 
excercise_contr_recovery_1 = contr_recovery_1[contr_recovery_1["participant"].isin(excercisers)] 
excercise_contr_recovery_2 = contr_recovery_2[contr_recovery_2["participant"].isin(excercisers)] 
excercise_contr_recovery_3 = contr_recovery_3[contr_recovery_3["participant"].isin(excercisers)] 
no_excercise_contr_before_run = contr_before_run[~contr_before_run["participant"].isin(excercisers)]
no_excercise_contr_recovery_0 = contr_recovery_0[~contr_recovery_0["participant"].isin(excercisers)]
no_excercise_contr_recovery_1 = contr_recovery_1[~contr_recovery_1["participant"].isin(excercisers)]
no_excercise_contr_recovery_2 = contr_recovery_2[~contr_recovery_2["participant"].isin(excercisers)]
no_excercise_contr_recovery_3 = contr_recovery_3[~contr_recovery_3["participant"].isin(excercisers)]
excercise_treat_before_run = treat_before_run[treat_before_run["participant"].isin(excercisers)]
excercise_treat_recovery_0 = treat_recovery_0[treat_recovery_0["participant"].isin(excercisers)]
excercise_treat_recovery_1 = treat_recovery_1[treat_recovery_1["participant"].isin(excercisers)]
excercise_treat_recovery_2 = treat_recovery_2[treat_recovery_2["participant"].isin(excercisers)]
excercise_treat_recovery_3 = treat_recovery_3[treat_recovery_3["participant"].isin(excercisers)]
no_excercise_treat_before_run = treat_before_run[~treat_before_run["participant"].isin(excercisers)]
no_excercise_treat_recovery_0 = treat_recovery_0[~treat_recovery_0["participant"].isin(excercisers)]
no_excercise_treat_recovery_1 = treat_recovery_1[~treat_recovery_1["participant"].isin(excercisers)]
no_excercise_treat_recovery_2 = treat_recovery_2[~treat_recovery_2["participant"].isin(excercisers)]
no_excercise_treat_recovery_3 = treat_recovery_3[~treat_recovery_3["participant"].isin(excercisers)]

male_contr_before_run = contr_before_run[contr_before_run["participant"].isin(males)] 
male_contr_recovery_0 = contr_recovery_0[contr_recovery_0["participant"].isin(males)] 
male_contr_recovery_1 = contr_recovery_1[contr_recovery_1["participant"].isin(males)] 
male_contr_recovery_2 = contr_recovery_2[contr_recovery_2["participant"].isin(males)] 
male_contr_recovery_3 = contr_recovery_3[contr_recovery_3["participant"].isin(males)] 
female_contr_before_run = contr_before_run[~contr_before_run["participant"].isin(males)]
female_contr_recovery_0 = contr_recovery_0[~contr_recovery_0["participant"].isin(males)]
female_contr_recovery_1 = contr_recovery_1[~contr_recovery_1["participant"].isin(males)]
female_contr_recovery_2 = contr_recovery_2[~contr_recovery_2["participant"].isin(males)]
female_contr_recovery_3 = contr_recovery_3[~contr_recovery_3["participant"].isin(males)]
male_treat_before_run = treat_before_run[treat_before_run["participant"].isin(males)]
male_treat_recovery_0 = treat_recovery_0[treat_recovery_0["participant"].isin(males)]
male_treat_recovery_1 = treat_recovery_1[treat_recovery_1["participant"].isin(males)]
male_treat_recovery_2 = treat_recovery_2[treat_recovery_2["participant"].isin(males)]
male_treat_recovery_3 = treat_recovery_3[treat_recovery_3["participant"].isin(males)]
female_treat_before_run = treat_before_run[~treat_before_run["participant"].isin(males)]
female_treat_recovery_0 = treat_recovery_0[~treat_recovery_0["participant"].isin(males)]
female_treat_recovery_1 = treat_recovery_1[~treat_recovery_1["participant"].isin(males)]
female_treat_recovery_2 = treat_recovery_2[~treat_recovery_2["participant"].isin(males)]
female_treat_recovery_3 = treat_recovery_3[~treat_recovery_3["participant"].isin(males)]

## HRV ah sheli

In [ ]:
import pandas as pd
import numpy as np

exp_data = exp_df.rename(columns={"bpm": "time"})

# Initialize an empty list to store results
hrv_results = []

# Loop through each participant
for participant in exp_data["participant"].unique():
    # Extract data for the current participant
    participant_data = exp_data[exp_data["participant"] == participant]
    
    # Sort by time to ensure correct intervals
    participant_data = participant_data.sort_values("time")

    # Calculate Inter-Beat Intervals (IBIs)
    ibi = np.diff(participant_data["time"].values)  # Time differences in seconds
    
    # Compute HRV metrics
    mean_ibi = np.mean(ibi) if len(ibi) > 0 else np.nan
    sdnn = np.std(ibi) if len(ibi) > 0 else np.nan
    rmssd = np.sqrt(np.mean(np.diff(ibi)**2)) if len(ibi) > 1 else np.nan
    pnn50 = np.sum(np.abs(np.diff(ibi)) > 0.05) / len(ibi) * 100 if len(ibi) > 1 else np.nan

    # Append results for the participant
    hrv_results.append({
        "participant": participant,
        "Mean IBI (s)": mean_ibi,
        "SDNN (s)": sdnn,
        "RMSSD (s)": rmssd,
        "pNN50 (%)": pnn50,
    })

# Convert HRV results to a DataFrame
hrv_df = pd.DataFrame(hrv_results).sort_values("participant")

# Display the HRV DataFrame to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="HRV Metrics Per Participant", dataframe=hrv_df)

* Mean IBI: Average time between beats.
* SDNN (Standard Deviation of NN intervals): Standard deviation of the IBIs.
* RMSSD (Root Mean Square of Successive Differences): A measure of short-term HRV.
* pNN50: Percentage of successive IBIs differing by more than 50 ms.

In [ ]:
hrv_df.head(2)

,participant,Mean IBI (s),SDNN (s),RMSSD (s),pNN50 (%)
0,1,0.637598,0.208824,0.231693,41.836735
10,2,0.509489,0.041386,0.007474,0.000000


## Merge HRV ah sheli and BPM

In [ ]:
contr_hrv = hrv_df[hrv_df["participant"].isin(contr_before_run["participant"].tolist())]
treat_hrv = hrv_df[hrv_df["participant"].isin(treat_before_run["participant"].tolist())]
smk_contr_hrv = hrv_df[hrv_df["participant"].isin(smk_contr_before_run["participant"].tolist())]
no_smk_contr_hrv = hrv_df[hrv_df["participant"].isin(no_smk_contr_before_run["participant"].tolist())]
smk_treat_hrv = hrv_df[hrv_df["participant"].isin(smk_treat_before_run["participant"].tolist())]
no_smk_treat_hrv = hrv_df[hrv_df["participant"].isin(no_smk_treat_before_run["participant"].tolist())]
excercise_contr_hrv = hrv_df[hrv_df["participant"].isin(excercise_contr_before_run["participant"].tolist())]
no_excercise_contr_hrv = hrv_df[hrv_df["participant"].isin(no_excercise_contr_before_run["participant"].tolist())]
excercise_treat_hrv = hrv_df[hrv_df["participant"].isin(excercise_treat_before_run["participant"].tolist())]
no_excercise_treat_hrv = hrv_df[hrv_df["participant"].isin(no_excercise_treat_before_run["participant"].tolist())]
male_contr_hrv = hrv_df[hrv_df["participant"].isin(male_contr_before_run["participant"].tolist())]
female_contr_hrv = hrv_df[hrv_df["participant"].isin(female_contr_before_run["participant"].tolist())]
male_treat_hrv = hrv_df[hrv_df["participant"].isin(male_treat_before_run["participant"].tolist())]
female_treat_hrv = hrv_df[hrv_df["participant"].isin(female_treat_before_run["participant"].tolist())]

## Save

In [13]:
import pandas as pd
import inspect

def save_dfs_to_csv():
    # Get the current global variables
    global_vars = globals()
    
    # Iterate over all variables in the global namespace
    for var_name, var_value in global_vars.items():
        # Check if the variable is a DataFrame
        if isinstance(var_value, pd.DataFrame):
            # Save the DataFrame to a CSV file named after the variable
            filename = f"./to_ella/{var_name}.csv"
            var_value.to_csv(filename, index=False)
            print(f"Saved {var_name} to {filename}")

In [15]:
# Call the function to save all DataFrames to CSV
save_dfs_to_csv()

Saved _ to ./to_ella/_.csv
Saved nivdakim_df to ./to_ella/nivdakim_df.csv
Saved unified_data to ./to_ella/unified_data.csv
Saved df to ./to_ella/df.csv
Saved bs_df to ./to_ella/bs_df.csv
Saved exp_df to ./to_ella/exp_df.csv
Saved contr_before_run to ./to_ella/contr_before_run.csv
Saved contr_recovery_0 to ./to_ella/contr_recovery_0.csv
Saved contr_recovery_1 to ./to_ella/contr_recovery_1.csv
Saved contr_recovery_2 to ./to_ella/contr_recovery_2.csv
Saved contr_recovery_3 to ./to_ella/contr_recovery_3.csv
Saved treat_before_run to ./to_ella/treat_before_run.csv
Saved treat_recovery_0 to ./to_ella/treat_recovery_0.csv
Saved treat_recovery_1 to ./to_ella/treat_recovery_1.csv
Saved treat_recovery_2 to ./to_ella/treat_recovery_2.csv
Saved treat_recovery_3 to ./to_ella/treat_recovery_3.csv
Saved smk_contr_before_run to ./to_ella/smk_contr_before_run.csv
Saved smk_contr_recovery_0 to ./to_ella/smk_contr_recovery_0.csv
Saved smk_contr_recovery_1 to ./to_ella/smk_contr_recovery_1.csv
Saved smk_c

## ignore

In [ ]:
def viz_bpm_hist(data: pd.Series, subtitle: str = "") -> None:   
    mean = data.mean()
    std = data.std()
    
    data.hist()
    plt.title(f"BPM Histogram\n{subtitle}")
    plt.xlabel("BPM")
    plt.ylabel("Frequency")
    plt.axvline(mean, color='red', linestyle='dashed', linewidth=2, label=f"Mean: {mean:.2f}")
    plt.axvline(mean + std, color='blue', linestyle='dashed', linewidth=1, label=f"Mean+Std: {mean+std:.2f}")
    plt.axvline(mean - std, color='blue', linestyle='dashed', linewidth=1, label=f"Mean-Std: {mean-std:.2f}")
    plt.legend()
    plt.show()